# Greeks Calculation and Analysis

This notebook demonstrates calculation of option Greeks using both analytical formulas and Monte Carlo methods.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qmc_options import generators, greeks, analytical

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Greeks from Black-Scholes (Analytical)

First, let's calculate Greeks using the analytical Black-Scholes formulas.

In [ ]:
# Option parameters
params = {
    'S0': 100.0,
    'K': 100.0,
    'r': 0.05,
    'delta': 0.02,
    'sigma': 0.25,
    'T': 1.0
}

# Calculate all Greeks
bs_delta = analytical.call_delta(**params)
bs_gamma = analytical.call_gamma(**params)
bs_vega = analytical.call_vega(**params)
bs_theta = analytical.call_theta(**params)
bs_rho = analytical.call_rho(**params)

print("Black-Scholes Greeks:")
print(f"Delta: {bs_delta:.4f}")
print(f"Gamma: {bs_gamma:.6f}")
print(f"Vega:  {bs_vega:.4f}")
print(f"Theta: {bs_theta:.4f}")
print(f"Rho:   {bs_rho:.4f}")

## 2. Monte Carlo Greeks - Pathwise Method

The pathwise derivative method works well for Greeks of smooth payoffs.

In [ ]:
# Generate QMC points
N = 50000
qmc_points = generators.halton([2], N)[:, 0]

# Calculate Greeks using pathwise method
mc_delta = greeks.pathwise_delta_european_call(points=qmc_points, **params)
mc_gamma = greeks.pathwise_gamma_european_call(points=qmc_points, **params)
mc_vega = greeks.pathwise_vega_european_call(points=qmc_points, **params)
mc_rho = greeks.pathwise_rho_european_call(points=qmc_points, **params)

print("\nMonte Carlo Greeks (Pathwise):")
print(f"Delta: {mc_delta:.4f} (Error: {abs(mc_delta - bs_delta):.6f})")
print(f"Gamma: {mc_gamma:.6f} (Error: {abs(mc_gamma - bs_gamma):.8f})")
print(f"Vega:  {mc_vega:.4f} (Error: {abs(mc_vega - bs_vega):.6f})")
print(f"Rho:   {mc_rho:.4f} (Error: {abs(mc_rho - bs_rho):.6f})")

## 3. Monte Carlo Greeks - Likelihood Ratio Method

The likelihood ratio (score function) method is more general but can have higher variance.

In [ ]:
# Calculate Greeks using likelihood ratio method
lr_delta = greeks.likelihood_delta_european_call(points=qmc_points, **params)
lr_gamma = greeks.likelihood_gamma_european_call(points=qmc_points, **params)
lr_vega = greeks.likelihood_vega_european_call(points=qmc_points, **params)
lr_theta = greeks.likelihood_theta_european_call(points=qmc_points, **params)
lr_rho = greeks.likelihood_rho_european_call(points=qmc_points, **params)

print("\nMonte Carlo Greeks (Likelihood Ratio):")
print(f"Delta: {lr_delta:.4f} (Error: {abs(lr_delta - bs_delta):.6f})")
print(f"Gamma: {lr_gamma:.6f} (Error: {abs(lr_gamma - bs_gamma):.8f})")
print(f"Vega:  {lr_vega:.4f} (Error: {abs(lr_vega - bs_vega):.6f})")
print(f"Theta: {lr_theta:.4f} (Error: {abs(lr_theta - bs_theta):.6f})")
print(f"Rho:   {lr_rho:.4f} (Error: {abs(lr_rho - bs_rho):.6f})")

## 4. Delta as a Function of Spot Price

Let's visualize how Delta changes with the spot price.

In [ ]:
# Range of spot prices
S_range = np.linspace(70, 130, 50)
deltas = []

for S in S_range:
    params_temp = params.copy()
    params_temp['S0'] = S
    delta = analytical.call_delta(**params_temp)
    deltas.append(delta)

plt.figure(figsize=(10, 6))
plt.plot(S_range, deltas, linewidth=2)
plt.axvline(params['K'], color='red', linestyle='--', label=f'Strike K={params["K"]}')
plt.axhline(0.5, color='gray', linestyle=':', alpha=0.5)
plt.xlabel('Spot Price (S)')
plt.ylabel('Delta')
plt.title('Call Option Delta vs Spot Price')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 5. Gamma Profile

Gamma measures the rate of change of Delta and is maximum at-the-money.

In [ ]:
# Calculate Gamma for different spot prices
gammas = []

for S in S_range:
    params_temp = params.copy()
    params_temp['S0'] = S
    gamma = analytical.call_gamma(**params_temp)
    gammas.append(gamma)

plt.figure(figsize=(10, 6))
plt.plot(S_range, gammas, linewidth=2, color='green')
plt.axvline(params['K'], color='red', linestyle='--', label=f'Strike K={params["K"]}')
plt.xlabel('Spot Price (S)')
plt.ylabel('Gamma')
plt.title('Call Option Gamma vs Spot Price')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print(f"\nGamma is maximum near the strike price (ATM)")

## 6. Time Decay (Theta)

Theta shows how option value changes as time passes.

In [ ]:
# Calculate option price and Theta for different times to maturity
T_range = np.linspace(0.1, 2.0, 50)
prices = []
thetas = []

for T in T_range:
    params_temp = params.copy()
    params_temp['T'] = T
    price = analytical.black_scholes_call(**params_temp)
    theta = analytical.call_theta(**params_temp)
    prices.append(price)
    thetas.append(theta)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(T_range, prices, linewidth=2)
ax1.set_xlabel('Time to Maturity (years)')
ax1.set_ylabel('Option Price ($)')
ax1.set_title('Option Price vs Time to Maturity')
ax1.grid(True, alpha=0.3)

ax2.plot(T_range, thetas, linewidth=2, color='orange')
ax2.axhline(0, color='black', linestyle='-', linewidth=0.5)
ax2.set_xlabel('Time to Maturity (years)')
ax2.set_ylabel('Theta')
ax2.set_title('Theta vs Time to Maturity')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Comparison Table

Let's create a summary table comparing analytical vs Monte Carlo Greeks.

In [ ]:
comparison = pd.DataFrame({
    'Greek': ['Delta', 'Gamma', 'Vega', 'Theta', 'Rho'],
    'Analytical': [
        f"{bs_delta:.4f}",
        f"{bs_gamma:.6f}",
        f"{bs_vega:.4f}",
        f"{bs_theta:.4f}",
        f"{bs_rho:.4f}"
    ],
    'MC Pathwise': [
        f"{mc_delta:.4f}",
        f"{mc_gamma:.6f}",
        f"{mc_vega:.4f}",
        'N/A',
        f"{mc_rho:.4f}"
    ],
    'MC Likelihood': [
        f"{lr_delta:.4f}",
        f"{lr_gamma:.6f}",
        f"{lr_vega:.4f}",
        f"{lr_theta:.4f}",
        f"{lr_rho:.4f}"
    ]
})

print("\nGreeks Comparison (N=50,000 samples):")
print(comparison.to_string(index=False))